# **ReAct RAG**

**ReAct**システムは、推論、行動、情報検索を組み合わせた高度な技術です。環境を分析し、関連するデータを検索し、その情報を用いて意思決定や応答の生成を行います。

この例では、3つのベクトルストアを用いて関連データを効率的に管理・検索する ReAct RAG システムを構築します。

- **Appointment (アポイントメント)：** 
  - 今後のアポイントメントのスケジュールや詳細情報を扱います。患者ID、受診理由、そして来院時の主な症状が記載され、アポイントメントに必要な基本情報。
- **Patient History (患者履歴)：**
  - 患者の包括的な医療履歴を保存し、検索できるようにします。患者の年齢、口腔衛生の習慣、過去の治療歴や感情面の記述が含まれており、患者の全体的な医療履歴として十分な情報を提供する。
- **Last Appointment Summary (最新アポイントメント概要)：** 
  - 直近のアポイントメントの概要として、受診日の情報、実施した処置内容、検査結果、そして具体的な指導内容が含まれており、システム上で迅速に概要を把握するのに適しています。

このシステムは、推論とデータ検索を組み合わせることで、正確で文脈を意識した応答を提供し、患者ケアやアポイントメントの管理に役立ちます。

### Azure AI Foundry で DeepSeek R1とPhi-4をデプロイして利用。(2025年2月21日現在で両方とも無料)
（openai系API料金が高いことと、google系は経験したのでMicrosoftやその他をためしたい。）

**DeepSeek-R1**:
- 総パラメータ数は6710億、コンテキスト長は128K。(distillution modelは除く)
- Azure AI Foundryを通じて安全に利用可能らしいです。
- 日本語で追加学習したモデルもすでに有るらしい -> Lightblue/DeepSeek-R1-Distill-Qwen-7B-Japanese
- 現在、混み合っているためかレスポンスが遅い、許容範囲外。

**Phi-4**:
- 総パラメータ数は140億、コンテキスト長は16K。
- Microsoftが2024年12月13日に発表した最新の小規模言語モデル（SLM）で、
- MicrosoftがこのほどHugging Face上で公式のウェイトをMITライセンス付きで公開しました。
- Phi-4は、日本語を含む多言語に対応しており、Azure AI Foundryを通じて利用可能です。
- レスポンス速度は許容範囲

[こちら](https://zenn.dev/givery_ai_lab/articles/f35586b839f64f#3.2-%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E4%BD%9C%E6%88%90)を参考に使ってみようと思います。


## 安全性 East-USのサーバー
Microsoftは「Azure AI Foundryの一部であるDeepSeek-R1は、信頼性が高く、企業は高度なAIをシームレスに統合しながら、セキュリティや責任あるAIのコミットメントを満たすことができます。これらはすべてMicrosoftの信頼性とイノベーションに支えられています」「DeepSeek-R1は、モデルの動作の自動評価や潜在的なリスクを軽減するための広範なセキュリティレビューなど、厳格なレッドチームによる安全性の評価を受けています。Azure AI Foundryでは、企業が自信を持ってAIソリューションをデプロイするための、安全でコンプライアンスに準拠した責任ある環境を提供します」

In [ ]:
! uv pip install -qU langchain-azure-ai
# ! uv pip install -qU azure-ai-inference 

# Chat Completion Test in DeepSeek-R1 and Phi-4

In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["AZURE_INFERENCE_CREDENTIAL"] = os.getenv("AZURE_API_KEY")
os.environ["AZURE_INFERENCE_ENDPOINT"] = os.getenv("AZURE_TARGET_URI")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

In [13]:
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

llm = AzureAIChatCompletionsModel(
    model_name="Phi-4",
    # model_name="DeepSeek-R1",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Japanese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)
ai_msg

私はプログラミングが大好きです。


AIMessage(content='私はプログラミングが大好きです。', additional_kwargs={}, response_metadata={'model': 'phi4', 'token_usage': {'input_tokens': 31, 'output_tokens': 15, 'total_tokens': 46}, 'finish_reason': 'stop'}, id='run-02e35952-a7d8-4842-8ec8-5d446fe8bbac-0', usage_metadata={'input_tokens': 31, 'output_tokens': 15, 'total_tokens': 46})

# 評価用データの準備

In [15]:
import pandas as pd
eval_df = pd.read_csv('../data/eval_data.csv')
eval_df.head(3)

,question,target_answer,target_page_no
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,13
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,16
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,21


# Embedding

In [22]:
# huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks... To disable this warning, you can either: - Avoid using tokenizers before the fork if possible - Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
os.environ["TOKENIZERS_PARALLELISM"] = "true" # 警告対策　tokenizersライブラリの並列処理を明示的にON 

# load embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}
)

2025-02-21 15:47:40,301 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: mps
2025-02-21 15:47:40,301 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: intfloat/multilingual-e5-base


In [23]:
# Load from local file
from langchain.vectorstores import FAISS
vectorstore = FAISS.load_local("../data/db/faiss_index", embeddings, allow_dangerous_deserialization=True) # これはpickleによるデシリアライゼーションの安全性に関する警告であり、自作のDBなど信頼できるソースからのファイルであれば True で問題ありません
# create retriever
retriever = vectorstore.as_retriever()

2025-02-21 15:53:57,271 | INFO | faiss.loader | Loading faiss.
2025-02-21 15:53:57,437 | INFO | faiss.loader | Successfully loaded faiss.
2025-02-21 15:53:57,442 | INFO | faiss | Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
